In [ ]:
import os
import pandas as pd
import numpy as np
import sys
# Add the root directory /workspaces/llm_etl to sys.path
sys.path.append(os.path.abspath(os.path.join('..', '..')))
# Now import your module
from spider2_utils import load_csv_database

-setup-

In [ ]:
import pandas as pd

_database = load_csv_database("Pagila", rows_limit=-1)

category = _database["category"]
film_category = _database["film_category"]
film = _database["film"]
inventory = _database["inventory"]
rental = _database["rental"]
customer = _database["customer"]
address = _database["address"]
city = _database["city"]

### Question
Please help me find the film category with the highest total rental hours in cities where the city's name either starts with \"A\" or contains a hyphen.

User Intent: Drop 'last_update' columns from all DataFrames if present


In [ ]:
for df in [category, film_category, film, inventory, rental, customer, address, city]:
    if 'last_update' in df.columns:
        df.drop(columns=['last_update'], inplace=True)

### User Intent 2: Join `category` and `film_category` using `category_id`

In [ ]:
cat_film_cat = pd.merge(category, film_category, on="category_id")

### User Intent 3: Join with `film` using `film_id`

In [ ]:
cat_film = pd.merge(cat_film_cat, film, on="film_id")

### User Intent 4: Join with `inventory` using `film_id`

In [ ]:
cat_film_inv = pd.merge(cat_film, inventory, on="film_id")

### User Intent 5: Join with `rental` using `inventory_id`

In [ ]:
cat_film_inv_rent = pd.merge(cat_film_inv, rental, on="inventory_id")

### User Intent 6: Join with `customer` using `customer_id`

In [ ]:
cat_rent_cust = pd.merge(cat_film_inv_rent, customer, on="customer_id")

### User Intent 7: Join with `address` using `address_id`

In [ ]:
cat_cust_addr = pd.merge(cat_rent_cust, address, on="address_id")

### User Intent 8: Join with `city` using `city_id`

In [ ]:
cat_addr_city = pd.merge(cat_cust_addr, city, on="city_id")

### User Intent 9: Filter rows where city name starts with 'a' or contains a hyphen

In [ ]:
cat_addr_city['city_lower'] = cat_addr_city['city'].str.lower()
filtered = cat_addr_city[
    (cat_addr_city['city_lower'].str.startswith('a')) | (cat_addr_city['city'].str.contains('-'))
]

### User Intent 10: Compute rental duration in hours for each rental

In [ ]:
filtered['rental_date'] = pd.to_datetime(filtered['rental_date'])
filtered['return_date'] = pd.to_datetime(filtered['return_date'])
filtered['rental_hours'] = (filtered['return_date'] - filtered['rental_date']).dt.total_seconds() / 3600

### User Intent 11: Group by category name and sum rental hours

In [ ]:
category_hours = filtered.groupby('name')['rental_hours'].sum().reset_index()

### User Intent 12: Sort by total rental hours descending and select the top category

In [ ]:
top_category = category_hours.sort_values(by='rental_hours', ascending=False).head(1)
print(top_category)